In [1]:
%matplotlib widget
import numpy as np
import matplotlib.pyplot as plt
import jupyros as jr
import rospy
from nav_msgs.msg import Odometry
from matplotlib import animation, rc
import rt2_assignment1.msg
from geometry_msgs.msg import Twist
from ipywidgets import Button, Layout, ButtonStyle, GridBox, VBox, HBox 
import ipywidgets as widgets
from rt2_assignment1.srv import Command
from IPython.display import display
from tf import transformations
import math

rospy.init_node('visualized_node')

# visualize the trajectory of the robot
- a line plot for visualizing cmd_vel vs. actual velocity (for linear and angular velocity) 
https://python.atelierkobato.com/annotate/
https://www.it-swarm-ja.com/ja/python/python-matplotlib%E3%82%A2%E3%83%8B%E3%83%A1%E3%83%BC%E3%82%B7%E3%83%A7%E3%83%B3%E3%81%A7%E5%81%9C%E6%AD%A2%E9%96%8B%E5%A7%8B%E4%B8%80%E6%99%82%E5%81%9C%E6%AD%A2/1073484243/

In [2]:
jr.live_plot("/cmd_vel/:linear.x:angular.z", Twist, 10000) 

Figure(axes=[Axis(label='X', scale=LinearScale()), Axis(label='Y', orientation='vertical', scale=LinearScale()…

- an xy graph showing the robot’s position and the orientation 

In [3]:
# First set up the figure and the axis 
fig, ax = plt.subplots()

ax.set_xlim(( -5, 5))
ax.set_ylim((-5, 5))


line, = ax.plot([], [], 'ro')
x_pos=0
y_pos=0
x_data=[]
y_data=[]
yaw=0
theta=[]
actual_linear_vel=[]
actual_angular_vel=[]
interation=0

def init():
    line.set_data([], [])
    return (line,)


def odom_callback(msg):
    global x_pos,y_pos,theta,actual_linear_vel,interation
    x_pos=msg.pose.pose.position.x
    print(x_pos)
    y_pos=msg.pose.pose.position.y
    y_data.append(y_pos)
    x_data.append(x_pos)
    if interation>110:
        dx=x_data[-1]-x_data[-10]
        dy=y_data[-1]-y_data[-10]
        
    else:
        dx=0
        dy=0
    interation+=1
    #linear_vel=math.sqrt(dx**2+dy**2)
    print("linear_vel"+str(linear_vel))
    actual_linear_vel=actual_linear_vel.append(np.sqrt(dx**2+dy**2))
    quaternion = (
        msg.pose.pose.orientation.x,
        msg.pose.pose.orientation.y,
        msg.pose.pose.orientation.z,
        msg.pose.pose.orientation.w)
    euler = transformations.euler_from_quaternion(quaternion)
    yaw = euler[2]
    theta.append(yaw)
    

def animate(i):
    global x_pos,y_pos,yaw
    line.set_data(x_data, y_data)
    arrow = plt.Arrow(x=x_pos,y=y_pos,dx=np.cos(yaw),dy=np.sin(yaw),width=0.01,head_width=0.05,head_length=0.2,length_includes_head=True,color='k')
    ax.add_patch(arrow)
    return (line, arrow)



jr.subscribe('/odom', Odometry, odom_callback)

# call the animator. blit=True means only re-draw the parts that have changed.

anim = animation.FuncAnimation(fig, animate, init_func=init,
                               frames=100, interval=20, blit=True)

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

- a line plot for visualizing cmd_vel vs. actual velocity (for linear and angular velocity)

In [4]:
fig1, ax1 = plt.subplots()

ax1.set_xlim(( 0, 100))
ax1.set_ylim((-1.2, 1.2))

linear_cmd_vel=[]
angular_cmd_vel=[]

time=[]

line1, = ax1.plot([], [], 'r-')
line2, = ax1.plot([], [], 'b-')
line3, = ax1.plot([], [], 'k*')
zero_time = rospy.get_time()

def init1():
    line1.set_data([], [])
    line2.set_data([], [])
    line3.set_data([], [])
    return (line1,line2,line3)

def twist_callback(msg):
    global linear_cmd_vel,angular_cmd_vel,zero_time,time
    linear_cmd_vel.append(msg.linear.x)
    angular_cmd_vel.append(msg.angular.z)
    time.append(rospy.get_time()-zero_time)

def animate1(i):
    global linear_cmd_vel,angular_cmd_vel,time,actual_linear_vel
    line1.set_data(time,linear_cmd_vel)
    line2.set_data(time, angular_cmd_vel)
    line3.set_data(time, actual_linear_vel)

    return (line1,line2,line3)

jr.subscribe('/cmd_vel', Twist, twist_callback)

anim = animation.FuncAnimation(fig1, animate1, init_func=init1,
                               frames=100, interval=20, blit=True)

[ERROR] [1622986300.444512, 1399.531000]: bad callback: <function odom_callback at 0x7fd410304310>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-3-4f85543967a2>", line 41, in odom_callback
    actual_linear_vel=actual_linear_vel.append(np.sqrt(dx**2+dy**2))
AttributeError: 'NoneType' object has no attribute 'append'



Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

[ERROR] [1622986300.551808, 1399.639000]: bad callback: <function odom_callback at 0x7fd410304310>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-3-4f85543967a2>", line 41, in odom_callback
    actual_linear_vel=actual_linear_vel.append(np.sqrt(dx**2+dy**2))
AttributeError: 'NoneType' object has no attribute 'append'

[ERROR] [1622986300.622442, 1399.723000]: bad callback: <function odom_callback at 0x7fd410304310>
Traceback (most recent call last):
  File "/opt/ros/noetic/lib/python3/dist-packages/rospy/topics.py", line 750, in _invoke_callback
    cb(msg)
  File "<ipython-input-3-4f85543967a2>", line 41, in odom_callback
    actual_linear_vel=actual_linear_vel.append(np.sqrt(dx**2+dy**2))
AttributeError: 'NoneType' object has no attribute 'append'

[ERROR] [1622986300.692267, 1399.786000]: bad callback: <function odom_callback at 0x7fd410304310>
Traceback (most rec